In [2]:
import stanza
stanza.download('en')
nlp = stanza.Pipeline('en')

2022-09-22 14:35:12 INFO: Downloading default packages for language: en (English)...
2022-09-22 14:35:13 INFO: File exists: C:\Users\Shubham Verma\stanza_resources\en\default.zip.
2022-09-22 14:35:16 INFO: Finished downloading models and saved to C:\Users\Shubham Verma\stanza_resources.
2022-09-22 14:35:16 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2022-09-22 14:35:16 INFO: Use device: cpu
2022-09-22 14:35:16 INFO: Loading: tokenize
2022-09-22 14:35:16 INFO: Loading: pos
2022-09-22 14:35:17 INFO: Loading: lemma
2022-09-22 14:35:17 INFO: Loading: depparse
2022-09-22 14:35:17 INFO: Loading: sentiment
2022-09-22 14:35:18 INFO: Loading: ner
2022-09-22 14:35:18 INFO: Done loading processors!


In [11]:
import pandas as pd

In [3]:
sent_aspects=[]
parag_aspects=[]

In [4]:
# aspect based opinion mining by splitting each row on '.'

def aspect_opinion_mining(para):
    sent_aspects=[]
    parag_aspects=[]
    for sen in para.split('.'): 
        # print('sen : ',sen)
        important = nlp(sen)
        descriptive_item = ''
        target = ''
        added_terms=''
        for sent in important.sentences:
            for wrd in sent.words:
                # print(wrd.text,wrd.pos,wrd.deprel)
                if ((wrd.deprel == 'root' and wrd.pos == 'NOUN') or (wrd.deprel == 'nsubj' and wrd.pos == 'NOUN') or(wrd.deprel == 'nsubj' and wrd.pos == 'PROPN')):
                    target = wrd.text
                if wrd.pos=='ADV' and wrd.deprel =='advmod':
                    added_terms = wrd.text
                if wrd.pos == 'ADJ':
                    descriptive_item = added_terms +' '+ wrd.text
                # if target!='' and wrd.pos=='NOUN' and wrd.deprel =='root':
                #     descriptive_item = added_terms +' '+ wrd.text                    
                if target!='' and descriptive_item!='':
                    sent_aspects.append({'aspect': target,'opinion': descriptive_item})
                    descriptive_item = ''
                    target = ''
                    added_terms=''
    if(sent_aspects!=[]):
        parag_aspects.append(sent_aspects)
        sent_aspects=[]
    return parag_aspects

In [5]:
parag_aspects = aspect_opinion_mining(" Good food. I w'd have liked more choices. Ambiance is nice")

In [6]:
parag_aspects

[[{'aspect': 'food', 'opinion': ' Good'},
  {'aspect': 'Ambiance', 'opinion': ' nice'}]]

In [9]:
def combine_aspect_opinion(parag_aspects):
    # joining aspects with opinions to create meaningful labels
    paragraph,sentence=[],[]
    for para in parag_aspects:
        for asp in para:
    #         print(asp)
            label = '-'.join(asp.values())
            sentence.append(label)
        paragraph.extend(sentence)
        sentence=[]
    combined_asp_op = ",".join(paragraph)
    return combined_asp_op

In [10]:
combine_aspect_opinion(parag_aspects)

'food- Good,Ambiance- nice'

In [12]:
df = pd.read_csv("C:\\Users\\Shubham Verma\\Project\\TextAnalytics\\Scripts\\aspect-based-opinion-mining\\2033_Julie_ Infection Control _Record No. UK-IC-01_AH_ W-TDI _UK_04102021 1415Hrs EST.csv")

In [15]:
df['aspect-opinion'] = [aspect_opinion_mining(sent) for sent in df.utter]

In [16]:
df.head()

,fname,spcode,seq,utter,stime,etime,splabel,key_phrase,aspect-opinion
0,2033_Julie_ Infection Control _Record No. UK-I...,B,1,"Alright, thank you. We'll start that the meeti...",0,12,moderator,none,[]
1,2033_Julie_ Infection Control _Record No. UK-I...,A,2,So currently I manufacture prevention and cont...,14,18,respondent,none,[]
2,2033_Julie_ Infection Control _Record No. UK-I...,B,3,So.,18,18,moderator,none,[]
3,2033_Julie_ Infection Control _Record No. UK-I...,A,4,"In a nutshell, my role is to help prevent pati...",21,60,respondent,none,[]
4,2033_Julie_ Infection Control _Record No. UK-I...,B,5,Alright. So how do your responsibilities relat...,61,66,moderator,none,[]


In [17]:
df.to_csv('aspect-opinion.csv')